In [49]:
#coding:utf-8
import pymongo
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import time
import sys
sys.path.append("./../../")
import common
import datetime
import matplotlib.pyplot as plt



# 得到某只新股的非一字涨停日期， 非一字涨停日状态（涨停 or 涨幅）， 非涨停收盘日， 非涨停收盘涨幅
# dates 为上市日
def get_stopDetail(stockid, dates, mysqls):
    global diOut_dict, backTen_dict, tenMonitor_dict
    stockid = '0'*(6-len(stockid)) + stockid

    edates = common.get_lastN_date(dates, -50)
    dates = common.format_date(dates,"%Y-%m-%d")
    edates = common.format_date(edates,"%Y-%m-%d")
    table="vmkt_equd"
    query = 'SELECT TICKER_SYMBOL, SEC_SHORT_NAME, TRADE_DATE, PRE_CLOSE_PRICE, OPEN_PRICE, HIGHEST_PRICE, LOWEST_PRICE, CLOSE_PRICE, ACT_PRE_CLOSE_PRICE ' \
                'from %s where TICKER_SYMBOL = "%s" and TRADE_DATE >= "%s" and TRADE_DATE <= "%s"' % (table, stockid, dates, edates)
    z = mysqls.dydb_query(query)
    z['actualRatio'] = (z['CLOSE_PRICE'].astype(np.float64) /z['PRE_CLOSE_PRICE'].astype(np.float64)) - 1
    z['actualRatio'] = (100*z['actualRatio']).round(2)

    z['priceTen'] = z['PRE_CLOSE_PRICE'].astype(np.float64) * 1.1
    z['priceTen'] = z['priceTen'].round(2)

    z['closeStatus'] = z['CLOSE_PRICE'] - z['priceTen']
    z['closeStatus'] = z['closeStatus'].apply(lambda x: True if x >= 0 else False)

    z['lowStatus'] = z['LOWEST_PRICE'] - z['priceTen']
    z['lowStatus'] = z['lowStatus'].apply(lambda x: True if x >= 0 else False)
    print stockid
    # 记录第一次的时间点
    # 如果当前还在涨停期间，跳过
    if len(z[z.lowStatus != True].index.values) == 0:
        return 100, "2018-01-01", "No Result", "INF", "2018-01-01", "100"
    print stockid
    
    first_low_out_index = z[z.lowStatus != True].index.values[0]
    first_low_out_day = z.loc[first_low_out_index, 'TRADE_DATE']
    
    # 当前还在回封期， 开板日设置为2018-01-01， 开板的index设置为无穷大
    if len(z[z.closeStatus != True].index.values) == 0:
        first_close_out_index = 10000
        first_close_out_day = "2018-01-01"
    else:
        first_close_out_index = z[z.closeStatus != True].index.values[0]
        first_close_out_day = z.loc[first_close_out_index, 'TRADE_DATE']

    if first_low_out_index == first_close_out_index:     # 开板没有回封
        diout_ratio = z.loc[first_close_out_index, 'actualRatio']
        diout_ratio = round(diout_ratio,2)
        if first_low_out_day not in diOut_dict.keys():
            diOut_dict[first_low_out_day] = {"stock":[stockid], "ratio":[diout_ratio]}
        else:
            diOut_dict[first_low_out_day]['stock'].append(stockid)
            diOut_dict[first_low_out_day]['ratio'].append(diout_ratio)
    else:    # 开板后回封
        diback_ratio = z.loc[first_low_out_index, 'actualRatio']
        diback_ratio = round(diback_ratio,2)
        # 记录每天的回封个数
        if first_low_out_day not in backTen_dict.keys():
            backTen_dict[first_low_out_day] = {"stock":[stockid], "ratio":[diback_ratio]}
        else:
            backTen_dict[first_low_out_day]['stock'].append(stockid)
            backTen_dict[first_low_out_day]['ratio'].append(diback_ratio)

        # 记录开板
        if first_close_out_index != 10000:
            diout_ratio = z.loc[first_close_out_index, 'actualRatio']
            diout_ratio = round(diout_ratio,2)
            if first_close_out_day not in diOut_dict.keys():
                diOut_dict[first_close_out_day] = {"stock":[stockid], "ratio":[diout_ratio]}
            else:
                diOut_dict[first_close_out_day]['stock'].append(stockid)
                diOut_dict[first_close_out_day]['ratio'].append(diout_ratio)
            
        
        # 记录每天回封后的涨幅（即：A日回封，B日不再涨停，记录A+1日到B日至今的涨幅）
        for idx in range(first_low_out_index+1, first_close_out_index+1):
            if idx > max(z.index.values):
                break
#                 idx = max(z.index.values)
            day = z.loc[idx, 'TRADE_DATE']
            
            
            
            
            
#             today = datetime.datetime.now()
#             print day, today
#             if day > datetime.date(today.year, today.month, today.day):
#                 break
            
            
            
            
            
            
            # 这里记录的是收盘涨幅，开板再回封的情况暂时不考虑
            ratio = z.loc[idx, 'actualRatio']
            if day not in tenMonitor_dict.keys():
                tenMonitor_dict[day] = {"stock":[stockid], "ratio":[ratio]}
            else:
                tenMonitor_dict[day]['stock'].append(stockid)
                tenMonitor_dict[day]['ratio'].append(ratio)
    
    ## 此处不全，diOut不仅仅是first_low_out_index， 还包括回封后开板的
    # return
    # 返回涨停数，开板日，开板状态（是否回封）， 非涨停日，非涨停日涨幅
    if first_close_out_index == 10000:
        actualRatio = 100
    else:
        actualRatio = z.loc[first_close_out_index, 'actualRatio']
    return first_close_out_index, first_low_out_day, first_close_out_index > first_low_out_index, first_close_out_index - first_low_out_index,first_close_out_day,actualRatio 



mongourl = "localhost"
mongodb = pymongo.MongoClient(mongourl)
results = mongodb.stock.ZDT_by_date.find({"date": {"$gte": "20160101"}})
count = 0
dframe = DataFrame()
mysqls = common.mysqldata()

global diOut_dict, backTen_dict, tenMonitor_dict
diOut_dict = {}    # 开板没有回封的记录
backTen_dict = {}  # 专门记录回封情况的dict
tenMonitor_dict = {}    # 回封后的每a日监控

for result in results:
    dates = result['date']
    if "Add_newStocks" in result.keys():
        stocks = result['Add_newStocks'].keys()
        for stock in stocks:
            dframe.loc[count, 'startDate'] = dates
            dframe.loc[count, 'stockid'] = stock
            dframe.loc[count, 'name'] = common.QueryStockMap(stock)[0]
            tmp_result = get_stopDetail(stock, dates, mysqls)
            (dframe.loc[count, 'numTen'], dframe.loc[count, 'openDate'], dframe.loc[count, 'backTen?'],
            dframe.loc[count, 'backDuring'], dframe.loc[count, 'stopDate'], dframe.loc[count, 'stopRatio']) = tmp_result
            count += 1

dframe.to_csv(u"新股开板情况.csv", encoding='GBK')

002778
002778
300499
300499
300500
300500
300501
300501
603377
603377
601900
601900
002788
002788
603608
603608
300502
300502
603027
603027
002790
002790
300503
300503
603520
603520
603919
603919
002789
002789
300505
300505
601020
601020
603861
603861
300484
300484
300506
300506
002792
002792
002791
002791
300474
300474
603028
603028
603798
603798
603701
603701
002793
002793
603868
603868
300508
300508
603726
603726
300509
300509
603029
603029
603822
603822
002795
002795
300507
300507
603528
603528
300511
300511
300510
300510
603101
603101
002796
002796
002797
002797
603779
603779
603959
603959
603339
603339
002798
002798
300512
300512
002800
002800
300513
300513
300516
300516
603737
603737
601611
601611
603131
603131
002799
002799
300515
300515
601127
601127
002801
002801
300519
300519
300518
300518
603909
603909
002802
002802
603958
603958
603016
603016
300521
300521
300522
300522
601966
601966
002805
002805
300520
300520
002803
002803
603069
603069
300517
300517
300525
300525
002806

In [4]:
# 存储下来dict
with open("./diOut_dict.txt", 'wb') as fHandler:
    fHandler.write(str(diOut_dict))

with open("./backTen_dict.txt", 'wb') as fHandler:
    fHandler.write(str(backTen_dict))

with open("./tenMonitor_dict.txt", 'wb') as fHandler:
    fHandler.write(str(tenMonitor_dict))

In [39]:
import datetime
datetime.datetime.now()

datetime.datetime(2016, 11, 7, 19, 24, 1, 448000)

In [25]:
# global diOut_dict, backTen_dict, tenMonitor_dict
# backTen_dict: key为回封当日
# diOut_dict: key为开板当日
# tenMonitor_dict: key为实时当日

In [50]:
dframe['startDate'] = dframe['startDate'].apply(lambda x: datetime.datetime.strptime(x, "%Y%m%d"))
# 记录1.1到11.7日，所有天对应的信息
# Index([u'loginNum', u'continuesNum', u'backStatus', u'openStocks',
#        u'openRatios', u'openNums', u'backTenStocks', u'backTenNums',
#        u'monitorStocks', u'monitorRatios'],
#       dtype='object')
# 当日这一批新股的上市数， 连板数，回封状态，当天的开板股票，开板涨幅，开板数，回封股票，回封数，监控中的股票，监控股票的涨幅
tframe = DataFrame()
total_date = common.getDate("20160101", "20161108")
for date in total_date:
    time_date = datetime.datetime.strptime(date, "%Y/%m/%d").date()
    if time_date in diOut_dict:  # 这一天有开板股票
        tframe.loc[time_date, 'openStocks'] = "_".join(diOut_dict[time_date]['stock'])
        tframe.loc[time_date, 'openRatios'] = "_".join([str(x) for x in diOut_dict[time_date]['ratio']])
        tframe.loc[time_date, 'openNums'] = len(diOut_dict[time_date]['stock'])
        
    if time_date in backTen_dict:  # 这一天有回封股票
        tframe.loc[time_date, 'backTenStocks'] = "_".join(backTen_dict[time_date]['stock'])
        tframe.loc[time_date, 'backTenNums'] = len(backTen_dict[time_date]['ratio'])
        
    if time_date in tenMonitor_dict:   # 这一天的监控股票
        tframe.loc[time_date, 'monitorStocks'] = "_".join(tenMonitor_dict[time_date]['stock'])
        tframe.loc[time_date, 'monitorRatios'] = "_".join([str(x) for x in tenMonitor_dict[time_date]['ratio']])
    
    # 该天上市的股票，连板个数
    targetFrame = dframe[dframe.startDate == time_date]
    if len(targetFrame) > 0:
        tframe.loc[time_date, 'continuesNum'] = list(targetFrame['numTen'])[0]
        tframe.loc[time_date, 'backStatus'] = list(targetFrame['backTen?'])[0]
    tframe.loc[time_date, 'loginNum'] = len(targetFrame)
    
tframe.to_csv(u'新股信息记录.csv', encoding='gbk')

In [1]:
# coding:utf-8
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 画图，看回封股票的收益率
tframe = pd.read_csv(u'新股信息记录.csv', encoding='gbk')
fig = plt.figure()
ax1 = fig.add_subplot(111)
for idx in tframe.index.values:
    m_ratios = str(tframe.loc[idx, 'monitorRatios'])
    if len(m_ratios) > 0:
        if "_" in m_ratios:
            values = m_ratios.split("_")
            values = [float(x) for x in values]
            for v in values:
                if v> 0 :
                    color = 'r'
                else:
                    color = 'g'
                ax1.plot(idx, v, color = color, marker = 'o')
        else:
            v = float(m_ratios)
            if v > 0:
                color = 'r'
            else:
                color = 'g'
            ax1.plot(idx, v, color = color, marker = 'o')
    else:
        ax1.plot(idx, 0, color = 'w')
point = 10
listLabels = tframe['Unnamed: 0']
step = len(tframe)/point
baseline = range(len(tframe)/step+1)
baseline = [int(x) * step for x in baseline]
if tframe.index.values[-1] not in baseline:
    baseline.append(tframe.index.values[-1])
if len(tframe) <= baseline[-1]:
    baseline = baseline[:-1]
ax1.set_xticks(baseline)
ax1.set_xticklabels(listLabels[baseline], rotation=30, fontsize=10)
ax1.set_xlim([min(tframe.index.values)-5, max(tframe.index.values)+5])
ax1.grid(True)
plt.savefig(u'开板回封收益.png', dpi = 300)
plt.show()
# 需要调查开板回封是负的原因，以及开板回封失败的股票

In [53]:
tframe

,Unnamed: 0,loginNum,continuesNum,backStatus,openStocks,openRatios,openNums,backTenStocks,backTenNums,monitorStocks,monitorRatios
0,2016-01-04,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-05,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-06,1.0,14.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-07,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-08,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016-01-11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016-01-12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016-01-13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016-01-14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016-01-15,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
x = "603977_603859_002819_002818_603667_300560_603888_603258_002817_002816_601882_603160_603060_603203_603556_300559_300558_300557_300556_300551_300550_300553_300552_603716_002819_603977"
x.split("_")

['603977',
 '603859',
 '002819',
 '002818',
 '603667',
 '300560',
 '603888',
 '603258',
 '002817',
 '002816',
 '601882',
 '603160',
 '603060',
 '603203',
 '603556',
 '300559',
 '300558',
 '300557',
 '300556',
 '300551',
 '300550',
 '300553',
 '300552',
 '603716',
 '002819',
 '603977']